## Scrape the Wiki Page and wrangle, clean, and read the data into a pandas dataframe

### Scraping the page

In [4]:
import requests
import pandas as pd

In [25]:
pip install lxml #use only if you get the error "lxml not found"

     |████████████████████████████████| 5.8MB 11.6MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipedia_page = requests.get(wikipedia_link)
wikipedia_doc = wikipedia_page.text

### Using BeautifulSoup to read the table

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(wikipedia_doc, 'html.parser')
table = soup.find('table', {'class': 'wikitable sortable'})
#table    

### Converting the table into a dataframe

In [7]:
col_names = ["PostalCode", "Borough", "Neighborhood"]
df = pd.read_html(str(table), skiprows=1)
df = pd.DataFrame.from_dict(df[0])
df.columns = col_names
df.head()

,PostalCode,Borough,Neighborhood
0,M2A,Not assigned,NaN
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"


### Removing cells with no Borough assigned

In [8]:
df = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Combining neighborhoods if there are more than one in a postal code

In [9]:
def combine_neighborhood(series):
    return series.str.cat(sep=', ')

df_by_pcode = df.groupby(["PostalCode", "Borough"])
df = df_by_pcode.agg({'Neighborhood': combine_neighborhood}).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### If there are no Neigborhoods assigned, Borough name will be used

In [10]:
def impute_neighborhood(row):
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
    
    return row

df = df.apply(impute_neighborhood, axis=1)
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Number of Rows

In [11]:
df.shape[0]

103

## Q2: Getting the latitude and the longitude coordinates of the neighborhoods

### Loading the geospatial coordinates from csv file

In [14]:
lat_lng_coords = pd.read_csv("Geospatial_Coordinates.csv")
lat_lng_coords.rename(columns={'Postal Code':'PostalCode'}, inplace=True) 
lat_lng_coords.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Combining the data frames

In [15]:
df_with_location = pd.merge(df, lat_lng_coords, how='inner', left_on='PostalCode', right_on='PostalCode')
df_with_location

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
